In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import src.data_handler as data_handler
import src.models as models
import os
import json

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))

CANTIDAD_DE_CLASES = 48
SEED = 42

X_images : np.ndarray[float] = np.load(f"{project_root}/TP03/data/X_images.npy")
y_images : np.ndarray[float] = np.load(f"{project_root}/TP03/data/y_images.npy")

y_images = np.array([[0 if y_images[x] != i else 1 for i in range(CANTIDAD_DE_CLASES)] for x in range(len(X_images))], dtype=float)
X_images = X_images / 255
X_train : pd.DataFrame
X_validation : pd.DataFrame
X_test : pd.DataFrame
X_train, X_validation, X_test, Y_train, Y_validation, Y_test = data_handler.get_splitted_dataset(pd.DataFrame(X_images), pd.DataFrame(y_images), seed=SEED)

np.random.seed(SEED)

model_score_03 : list[dict] = {}
with open("model_score_03.json", "r") as file:
    model_score_03 = json.load(file)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Parámetros
M = model_score_03[3]['M']
h = model_score_03[3]['h']
epochs = 500
learning_rate = model_score_03[3]['lr_range']
batch_size_2_pow = model_score_03[3]['batch_size_2']
K = model_score_03[3]['K']
c = model_score_03[3]['c'] if model_score_03[3]['S'] != 0 else 0
S = model_score_03[3]['S']
use_adam = model_score_03[3]['use_adam']
b1, b2 = model_score_03[3]['b1_b2']
L2 = model_score_03[3]['l2']
print_results_rate = 50

# Datos
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
Y_train_tensor = torch.tensor(Y_train, dtype=torch.float32)
dataset = TensorDataset(X_train_tensor, Y_train_tensor)
batch_size = 2 ** batch_size_2_pow
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Red Neuronal
layers = []
input_dim = M[0]
for units in h:
    layers.append(nn.Linear(input_dim, units))
    layers.append(nn.ReLU())
    input_dim = units
layers.append(nn.Linear(input_dim, M[1]))
network = nn.Sequential(*layers)

# Optimizer
optimizer = optim.Adam(network.parameters(), lr=learning_rate, betas=(b1, b2)) if use_adam else optim.SGD(network.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

# Entrenamiento
for epoch in range(epochs):
    for X_batch, Y_batch in dataloader:
        optimizer.zero_grad()
        outputs = network(X_batch)
        loss = criterion(outputs, Y_batch)
        if L2 != 0:
            l2_reg = torch.tensor(0., requires_grad=True)
            for param in network.parameters():
                l2_reg += torch.norm(param)
            loss += L2 * l2_reg
        loss.backward()
        optimizer.step()
    if epoch % print_results_rate == 0:
        print(f"Epoch {epoch}/{epochs} - Loss: {loss.item()}")
